# Lab 07 · RAG with Citations

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- Retrieved chunks are combined into grounded answers.
- Inline citation markers such as [S1] are emitted.
- Refusals are documented when no evidence is present.

## What will be learned
- Response assembly with citations is structured.
- Evidence gating ensures unsupported answers are refused.
- Backend orchestration for RAG is reinforced.

## Prerequisites & install
The following commands are intended for local execution.

```bash
cd ai-web/backend
. .venv/bin/activate
pip install google-generativeai
```

## Step-by-step tasks
### Step 1: RAG helper
A helper combines retrieved chunks with a Gemini completion.

In [ ]:
from pathlib import Path
rag_path = Path("ai-web/backend/app/rag.py")
rag_path.write_text('''from typing import List

from .vector import search
from .llm import chat


def answer(question: str) -> dict:
    retrieved = search(question, 3)
    if not retrieved:
        return {"answer": "No supported answer can be provided without evidence.", "chunks": []}
    citations = [f"[S{idx + 1}]" for idx in range(len(retrieved))]
    prompt = [
        {"role": "user", "content": f"Question: {question}. Use only the provided snippets."},
        {"role": "model", "content": "Sources:
" + "
".join(f"[S{idx + 1}] {text}" for idx, (text, _) in enumerate(retrieved))}
    ]
    completion = chat(prompt)
    return {"answer": completion, "chunks": [{"id": f"S{idx + 1}", "text": text, "score": score} for idx, (text, score) in enumerate(retrieved)], "citations": citations}
''')
print("RAG helper was created.")

### Step 2: Endpoint exposure
The RAG helper is surfaced under /api/answer.

In [ ]:
from pathlib import Path
main_path = Path("ai-web/backend/app/main.py")
text = main_path.read_text()
if "answer_endpoint" not in text:
    addition = '''
from .rag import answer as answer_question


@app.get("/api/answer")
def answer_endpoint(q: str):
    result = answer_question(q)
    if not result.get("chunks"):
        return {"answer": "No supported answer can be provided without evidence.", "citations": []}
    return result
'''
    main_path.write_text(text.rstrip() + "
" + addition)
    print("Answer endpoint was appended.")
else:
    print("Answer endpoint already present.")

## Validation / acceptance checks
```bash
# locally
curl 'http://localhost:8000/api/answer?q=course goals'
```
- A cited answer referencing [S1] style markers is produced when evidence exists.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- Citation rendering is enhanced in the frontend chat UI.
- Fallback messaging is drafted for unanswered questions.